### Importing libraries

In [1]:
import cv2 as cv
import numpy as np
import HandTrackingModule as htm
import time
import math

# from pycaw to adjust the volume
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume

In [2]:
wCam, hCam = 640, 480

cap = cv.VideoCapture(0)
cap.set(3, wCam)
cap.set(4, hCam)

True

**Creating an object of the class (from the module we created to detect hands)**

In [3]:
detector = htm.handDetector(detectionCon=0, maxHands=1)

In [4]:
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(
    IAudioEndpointVolume._iid_, CLSCTX_ALL, None
)
volume = cast(interface, POINTER(IAudioEndpointVolume))

volRange = volume.GetVolumeRange()
minVol = volRange[0]
maxVol = volRange[1]
vol=0
volBar=400
volPerc=0
area=0
colorVol = (255, 0, 0)

In [5]:
while True:
    success, img = cap.read()

    # Finding hands
    img = detector.findHands(img)
    lmList, bbox = detector.findPosition(img, draw=True)
    if len(lmList) != 0 :

        # Filtering based on the size of the hand
        area = (bbox[2]-bbox[0])*(bbox[3]-bbox[1])//100
        
        if 250 < area < 1000: # tackles the problem of varying distance of the hand from the screen problem

            # Distance between the index finger and the thumb
            length, img, lineInfo = detector.findDistance(4, 8, img)
            
            # Finger distance range: 50 to 300
            # Volume Range: -65 to 0
            # Converting volume
            volBar = np.interp(length, [50, 300], [400, 150]) 
            volPerc = np.interp(length, [50, 300], [0, 100])

            # Reducing resolution to make it smoother
            smoothness = 10
            volPerc = smoothness * round(volPerc / smoothness)

            # Check if the fingers are up
            fingers = detector.fingersUp()

            # Setting the volume if the pinky finger is down
            if not fingers[4]:
                volume.SetMasterVolumeLevelScalar(volPerc / 100, None)
                cv.circle(img, (lineInfo[4], lineInfo[5]), 15, (0, 255, 0), cv.FILLED)
                colorVol = (0, 255, 0)
            else:
                colorVol = (255, 0, 0)

    # Drawings
    cv.rectangle(img, (50, 150), (85, 400), (255, 0, 0), 3)
    cv.rectangle(img, (50, int(volBar)), (85, 400), (255, 0, 0), cv.FILLED)
    cv.putText(img, f'{int(volPerc)} %', (40, 450), cv.FONT_HERSHEY_COMPLEX, 1, (255, 0, 0), 3)
    cVol = int(volume.GetMasterVolumeLevelScalar() * 100)
    cv.putText(img, f'Vol Set: {int(cVol)}', (400, 50), cv.FONT_HERSHEY_COMPLEX, 1, colorVol, 3)

    
    cv.imshow("Image", img)
    cv.waitKey(20)

C:\Users\jayme\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


KeyboardInterrupt: 